### Using ground truth images

This notebook uses ground truth images to evaluate segmentation accuracy.

In [88]:
#setup libraries
import os, re, sys, glob, shutil, math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, filters, util, segmentation, morphology, measure
from cellpose import plot
from scipy import stats

#use_GPU = models.use_gpu()
#print('>>> GPU activated? %d'%use_GPU)

In [89]:
pipeline_name = 'CL'
ch1_name = 'NR'
ch2_name = 'CC'
data_path = '/home/exacloud/gscratch/HeiserLab/images/'

Read the names of the ground truth files, parse them to be able to identify the corresponding segmented files and store in a dataframe.

In [102]:
minutes_between_images = 30
path_to_ground_truth = '/home/exacloud/gscratch/HeiserLab/software/image_analysis_pipelines/AU565/ground_truth/'
ground_truth_data_paths =glob.glob(os.path.join(path_to_ground_truth,"*groundtruth.tif"))
df_path = pd.DataFrame(ground_truth_data_paths, columns =['path']).sort_values(by='path', ignore_index = True)
#parse ground truth filenames to get filenames of segmentation masks
df = df_path.path.str.extract(r'(?P<plateID>[A-Z]{2}[0-9]{5})_(?P<well>[A-D][1-6])_(?P<field>[1-6])_(?P<slice>[0-9]+)_')
df = df.astype({'slice': 'int32'})
df['elapsed_minutes'] = (df.slice-1)*minutes_between_images #assumes time slice numbering starts at 1
df['day'] = np.floor(df.elapsed_minutes/(24*60)).astype(int)
df['hour'] = np.floor((df.elapsed_minutes-df.day*(24*60))/60).astype(int)
df['minute'] = np.floor(df.elapsed_minutes-df.day*(24*60)-df.hour*60).astype(int)
df.day = df.day.apply(lambda x: '{0:0>2}'.format(x))
df.hour = df.hour.apply(lambda x: '{0:0>2}'.format(x))
df.minute = df.minute.apply(lambda x: '{0:0>2}'.format(x))
df['time_slice'] = df.day+"d"+df.hour+"h"+df.minute+"m"
df['mask_slice'] = df.slice-1
df['ground_truth_path'] = df_path.path
df['mask_path'] = data_path+df.plateID+"/Analysis/PI/intermediate_files/masks/"+df.well+"/field_"+df.field+"/"+df.well+"_"+df.field+"_image"+df.mask_slice.astype("str")+"_nuclei_masks.tif"

Loop through each row of the dataframe and compare the ilastik segmented and ground truth masks.

In [111]:
df['dice'] = 0.0
for index, image in df.iterrows():
    #load the ground truth masks and convert to binary
    ground_truth_mask = io.imread(image.ground_truth_path) > 0
    #load the segmented mask and convertto binary
    segmented_mask = io.imread(image.mask_path) > 0
    dice = np.sum(segmented_mask[ground_truth_mask])*2/(np.sum(segmented_mask) + np.sum(ground_truth_mask))
    df.loc[index,"dice"] = dice
    #create images that show the differences in the masks
    common_pixels = np.logical_and(ground_truth_mask, segmented_mask)
    ground_truth_unique_pixels = np.logical_xor(ground_truth_mask, common_pixels)
    segmented_unique_pixels = np.logical_xor(segmented_mask, common_pixels)
    
    path_name_p = os.path.join(data_path,image.plateID,image.well,"field_"+image.field,image.plateID+"_P_"+image.well+"_"+image.field+"_"+image.time_slice+".tif")
    path_name_r = os.path.join(data_path,image.plateID,image.well,"field_"+image.field,image.plateID+"_R_"+image.well+"_"+image.field+"_"+image.time_slice+".tif")

    # load the phase and red images for display purposes
    image_p = io.imread(path_name_p)
    #Create ground truth mask overlay and save it
    masks_o = plot.mask_overlay(image_p, ground_truth_mask, colors=None)
    ground_truth_mask_filename = image.ground_truth_path.replace(".tif", "_masks.png")
    ground_truth_mask_filename = ground_truth_mask_filename.replace("/ground_truth/", "/ground_truth/comparisons/")
    io.imsave(ground_truth_mask_filename, masks_o)
    #Create ground truth unique mask overlay and save it
    masks_o = plot.mask_overlay(image_p, ground_truth_unique_pixels, colors=None)
    ground_truth_unique_mask_filename = ground_truth_mask_filename.replace("_groundtruth_masks.png", "_groundtruth_unique_masks.png")
    io.imsave(ground_truth_unique_mask_filename, masks_o)
    #Create unique segmentation mask overlay and save it
    masks_o = plot.mask_overlay(image_p, segmented_unique_pixels, colors=None)
    segmented_unique_mask_filename = ground_truth_mask_filename.replace("_groundtruth_masks.png", "_segmented_unique_masks.png")
    io.imsave(segmented_unique_mask_filename, masks_o)
    #Create overlay of segmentation masks on phase save it
    masks_o = plot.mask_overlay(image_p, segmented_mask, colors=None)
    segmented_mask_filename = ground_truth_mask_filename.replace("_groundtruth_masks.png", "_segmented_masks.png")
    io.imsave(segmented_mask_filename, masks_o)
    #Combine common and both unique pixel images, save as RGB
    combined_filename = ground_truth_unique_mask_filename.replace("_groundtruth_unique_masks.png", "_combined.png")
    image_stack = np.dstack([segmented_unique_pixels, ground_truth_unique_pixels, common_pixels])
    io.imsave(combined_filename, util.img_as_ubyte(image_stack))
